In [1]:
import cudf as pd

In [2]:
# import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
def get_soup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.content,'html.parser')
    return soup,r

In [39]:
def get_anime_details(soup,url):
    details = {}
    details['url']=url
    details['name'] = soup.find(class_='title-name').text
    details['description'] = soup.find(itemprop='description').text
    try:
        details['score']=soup.find(itemprop='ratingValue').text
        info_list = [i.text.replace('\n','').replace(' ','') for i in 
                     soup.find(class_='leftside').find_all(class_='spaceit_pad')]
    
        cols = ['Type', 'Episodes','Premiered','Studios', 'Source', 
                'Rating','Members', 'Favorites']
    
        for col in cols:
            details[col.lower()] = next((element.split(':')[-1] for element 
                                 in info_list if col in element),None)
        try:
            genres = [i[:len(i)//2] for i in next((element.split(':')[-1] for element 
                                     in info_list if 'Genres' in element),None).split(',')]
            for i in range(len(genres)):
                details[f'genre-{i+1}'] = genres[i]
        except:
            genre = next((element.split(':')[-1] for element 
                                     in info_list if 'Genre' in element),None)
            details['genre-1'] = genre[:len(genre)//2]
    except:
        pass
    return pd.DataFrame(details,index=[0])

In [38]:
def get_review_detail(review_soup):
    details = {}
    details['username'] = review_soup.find(class_='username').text.replace('\n','')
    details['anime_name'] = review_soup.find(class_='title').text
    details['anime_link'] = review_soup.find(class_='title')['href']
    details['rating_user'] = review_soup.find(class_='rating').find(class_='num').text
    try:
        details['recommended'] = review_soup.find(class_='tag').text
    except:
        details['recommended'] = 'Recommended' if int(details['rating_user'])>=6 else 'Not Recommended'
    details['timestamp'] = review_soup.find(class_='update_at').text
    return pd.DataFrame(details,index=[0])

In [189]:
pages=2
offset=1
df = pd.DataFrame()
for page in range(offset,pages+offset):
    url = f'https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=2&preliminary=off&spoiler=on&p={page}'
    print(f'currently on page {url}')
    soup,_ = get_soup(url)
    all_reviews = soup.find_all(class_='review-element js-review-element')
    for review in all_reviews:
        df = pd.concat([df,get_review_detail(review)],ignore_index=True)
df.to_csv(f'data/review_{offset}-{offset+pages-1}.csv',index=False)

currently on page https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=2&preliminary=off&spoiler=on&p=1
currently on page https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=2&preliminary=off&spoiler=on&p=2


In [40]:
df_anime = pd.DataFrame()
for anime_link in df['anime_link'].unique().to_arrow().to_pylist():
    print(f'current anime {anime_link}')
    soup,_=get_soup(anime_link)
    df_anime = pd.concat([df_anime,get_anime_details(soup,anime_link)],ignore_index=True)
# len(df['anime_link'].unique().to_arrow())
df_anime.info()

current anime https://myanimelist.net/anime/1/Cowboy_Bebop
current anime https://myanimelist.net/anime/100/Shin_Shirayuki-hime_Densetsu_Prétear
current anime https://myanimelist.net/anime/1000/Uchuu_Kaizoku_Captain_Herlock
current anime https://myanimelist.net/anime/10005/Tetsujin_28-gou__Hakuchuu_no_Zangetsu
current anime https://myanimelist.net/anime/10012/Carnival_Phantasm
current anime https://myanimelist.net/anime/10015/Yu☆Gi☆Oh_Zexal
current anime https://myanimelist.net/anime/1002/Top_wo_Nerae_2_Diebuster
current anime https://myanimelist.net/anime/10020/Ore_no_Imouto_ga_Konnani_Kawaii_Wake_ga_Nai_Specials
current anime https://myanimelist.net/anime/10029/Coquelicot-zaka_kara
current anime https://myanimelist.net/anime/10030/Bakuman_2nd_Season
current anime https://myanimelist.net/anime/10033/Toriko
current anime https://myanimelist.net/anime/10036/Boku_no_Chikyuu_wo_Mamotte__Kiniro_no_Toki_Nagarete
current anime https://myanimelist.net/anime/1004/Kanojo_to_Kanojo_no_Neko
curren

In [41]:
df_anime.to_csv('data/anime.csv',index=False)

In [123]:
url = f'https://myanimelist.net/topanime.php?limit={limit}'

In [91]:
limit = 0

In [103]:
animes_soup = soup.find_all(class_='ranking-list')
anime_links = [anime_soup.find(class_='hoverinfo_trigger')['href'] for anime_soup in animes_soup]

In [115]:
df = pd.DataFrame()
for anime_link in anime_links[:3]:
    soup,_ = get_soup(anime_link)
